In [153]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [154]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
gsub=pd.read_csv('gender_submission.csv')

In [155]:
train.head()
train.describe(include='all')
#train.isnull().any()
#train.shape

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Shorney, Mr. Charles Joseph",male,NaN,NaN,NaN,CA. 2343,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [156]:
test.head()
gsub.head()
gsub.shape

(418, 2)

replacing NaN values in Age to a mean value

In [157]:
age_avg=train['Age'].mean(axis=0)
train['Age'].replace(np.nan,age_avg,inplace=True)

replacing NaN values in Cabin to the top value

In [158]:
train['Cabin'].replace(np.nan,'G6',inplace=True)

replacing NaN values in Embarked by a top value

In [159]:
train['Embarked'].replace(np.nan,'S',inplace=True)

dropping useless columns from the dataframe

In [160]:
del train['Name']

In [161]:
del train['Ticket']

In [162]:
ds=train[['Survived','Cabin']].groupby(by='Cabin',as_index=False).count()
ds.describe()

,Survived
count,147.000000
mean,6.061224
std,56.883108
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,691.000000


In [163]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,G6,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,G6,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,G6,S


In [164]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [165]:
gsub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


one hot encoding of the dataset

In [166]:
newds=train

In [167]:
sexds=pd.get_dummies(train['Sex'])
newds['Female']=sexds['female']
newds['Male']=sexds['male']
embarkds=pd.get_dummies(train['Embarked'])
newds[embarkds.columns]=embarkds
del newds['Sex']
del newds['Cabin']
del newds['Embarked']
newds.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Female,Male,C,Q,S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [168]:
newds.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Female           uint8
Male             uint8
C                uint8
Q                uint8
S                uint8
dtype: object

In [169]:
titanicForest=RandomForestClassifier(n_estimators=10,criterion='entropy')

In [170]:
from sklearn.cross_validation import train_test_split
X=train.drop(newds.columns[[0,1,6]],axis=1).values
y=newds['Survived']
type(X)
type(y)

pandas.core.series.Series

In [171]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=7)

In [172]:
titanicForest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [173]:
titanicpred=titanicForest.predict(x_test)

In [174]:
print(titanicpred[0:20])
print(y_test[0:20])

[0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0]
725    0
861    0
528    0
46     0
627    1
357    0
890    0
179    0
562    0
879    1
53     1
159    0
98     1
237    1
814    0
778    0
60     0
732    0
616    0
228    0
Name: Survived, dtype: int64


In [175]:
from sklearn import metrics

In [176]:
print(metrics.accuracy_score(y_test,titanicpred))

0.737430167598


using entire training dataset for training

In [177]:
newtestds=test
newtestds.head()
newtestds['Age'].replace(np.nan,newtestds['Age'].mean(axis=0),inplace=True)
newtestds.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

In [178]:
del newtestds['Name']
del newtestds['Ticket']

In [179]:
sexds2=pd.get_dummies(newtestds['Sex'])
newtestds['Female']=sexds['female']
newtestds['Male']=sexds['male']
embarkds=pd.get_dummies(newtestds['Embarked'])
newtestds[embarkds.columns]=embarkds
del newtestds['Sex']
del newtestds['Cabin']
del newtestds['Embarked']
newtestds.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Female,Male,C,Q,S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,0,1,0
3,895,3,27.0,0,0,8.6625,1,0,0,0,1
4,896,3,22.0,1,1,12.2875,0,1,0,0,1


In [190]:
titanicForest.fit(X,y)
print(X[0])

[  3.  22.   1.   0.   0.   1.   0.   0.   1.]


In [194]:

newtestds.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Female,Male,C,Q,S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,0,1,0
3,895,3,27.0,0,0,8.6625,1,0,0,0,1
4,896,3,22.0,1,1,12.2875,0,1,0,0,1


In [196]:
newx_test=newtestds.drop(newtestds.columns[[0,5]],axis=1).values
newy_test=gsub['Survived']
print(newx_test[0])
titanicpred2=titanicForest.predict(newy_test.reshape(1,-1))

[  3.   34.5   0.    0.    0.    1.    0.    1.    0. ]


/Users/abhilashsk/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


ValueError: Number of features of the model must match the input. Model n_features is 9 and input n_features is 418 

# test and train with entire training dataset

In [384]:
xtestds=pd.read_csv('test.csv')
trainds=pd.read_csv('train.csv')
ytestds=pd.read_csv('gender_submission.csv')

In [385]:
xtestds['Age'].replace(np.nan,xtestds['Age'].mean(axis=0),inplace=True)
xtestds['Fare'].replace(np.nan,xtestds['Fare'].mean(axis=0),inplace=True)
xtestds.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked       False
dtype: bool

In [386]:
trainds['Age'].replace(np.nan,trainds['Age'].mean(axis=0),inplace=True)
trainds['Embarked'].replace(np.nan,'S',inplace=True)
trainds.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked       False
dtype: bool

In [387]:
ds1=pd.get_dummies(xtestds['Sex'])
ds2=pd.get_dummies(xtestds['Cabin'])
ds3=pd.get_dummies(xtestds['Embarked'])
xtestds[ds1.columns]=ds1
xtestds[ds3.columns]=ds3
xtestds.drop(['Sex','Cabin','Embarked','Name','Ticket'],inplace=True,axis=1)
xtestds.head()
#ds2.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1


In [388]:
ds1=pd.get_dummies(trainds['Sex'])
ds2=pd.get_dummies(trainds['Cabin'])
ds3=pd.get_dummies(trainds['Embarked'])
trainds[ds1.columns]=ds1
trainds[ds3.columns]=ds3
trainds.drop(['Sex','Cabin','Embarked','Name','Ticket'],inplace=True,axis=1)
trainds.head()
#ds2.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [389]:
x_train=trainds.drop(['PassengerId','Survived'],axis=1)
y_train=trainds['Survived']
x_test=xtestds.drop(['PassengerId'],axis=1)
y_test=ytestds['Survived']
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(891, 10)
(891,)
(418, 10)
(418,)


In [390]:
for i in range(10,30):
    titanicForest2=RandomForestClassifier(n_estimators=i,criterion='entropy')
    titanicForest2.fit(x_train,y_train)
    titanicpred2=titanicForest2.predict(x_test)
    print(i,'  ',metrics.accuracy_score(y_test,titanicpred2))

10    0.796650717703
11    0.803827751196
12    0.827751196172
13    0.799043062201
14    0.818181818182
15    0.806220095694
16    0.803827751196
17    0.799043062201
18    0.791866028708
19    0.787081339713
20    0.820574162679
21    0.813397129187
22    0.803827751196
23    0.806220095694
24    0.806220095694
25    0.811004784689
26    0.799043062201
27    0.799043062201
28    0.799043062201
29    0.813397129187


In [391]:
titanicForest2=RandomForestClassifier(n_estimators=15,criterion='entropy')

In [392]:
titanicForest2.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [393]:
titanicpred2=titanicForest2.predict(x_test)

In [394]:
print(metrics.accuracy_score(y_test,titanicpred2))

0.796650717703


In [355]:
resultds=pd.DataFrame(titanicpred2)
resultds['PassengerId']=ytestds['PassengerId']
del resultds[resultds.columns[0]]
addds=pd.DataFrame(titanicpred2)
resultds['Survived']=addds
resultds.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [356]:
resultds.to_csv('abhilash_submission.csv')

In [295]:
ytestds.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [395]:
from sklearn.neighbors import KNeighborsClassifier

In [396]:
#for i in range(1,20):
knn=KNeighborsClassifier(n_neighbors=11)
knn.fit(x_train,y_train)
knnpred=knn.predict(x_test)
print(metrics.accuracy_score(y_test,knnpred))

0.681818181818


In [397]:
resultds=pd.DataFrame(knnpred)
resultds['PassengerId']=ytestds['PassengerId']
del resultds[resultds.columns[0]]
addds=pd.DataFrame(knnpred)
resultds['Survived']=addds
resultds.head()
resultds.to_csv('abhilash_submissionz.csv')

In [398]:
from sklearn.svm import SVC

In [399]:
svc=SVC(kernel="linear", C=0.025)
svc.fit(x_train,y_train)
svcpred=svc.predict(x_test)
print(metrics.accuracy_score(y_test,svcpred))

1.0


In [376]:
resultds=pd.DataFrame(svcpred)
resultds['PassengerId']=ytestds['PassengerId']
del resultds[resultds.columns[0]]
addds=pd.DataFrame(svcpred)
resultds['Survived']=addds
resultds.head()
resultds.to_csv('svc_submissionz.csv')

In [370]:
from sklearn.tree import DecisionTreeClassifier

In [401]:
dtc=DecisionTreeClassifier(max_depth=10)
dtc.fit(x_train,y_train)
dtcpred=dtc.predict(x_test)
print(metrics.accuracy_score(y_test,dtcpred))

0.815789473684
